
# Exact matches model

1. **For each training set sequence,  for each disease class, run Fisher’s exact test on 2x2 contingency table** counting the number of unique participants that are/aren't in a disease class and have/don't have a particular V/J/CDR3.


```
Contigency table is:
---------------------------------------------|---------------------- |
is other class |                                                     |
----------------      # unique participants with this property       -
is this class  |                                                     |
---------------|-----------------------------|---------------------- |
               | does not have this sequence | has this sequence --- |
```

This returns p-values representing how enriched each sequence is in each disease class.

No multiple hypothesis correction, but we do try some low p-value thresholds below that may simulate Bonferroni correction.

2. **Given a p-value threshold, featurize and fit a model.**

Featurization into `n_specimens x n_diseases` feature matrix:
- For a specimen, count how many of its sequences are exact matches to [sequences whose Covid-19 specificity p-values <= p-value threshold], [sequences whose HIV specificity p-values <= p-value threshold], etc.
- Divide by total number of sequences in the specimen




3. **Evaluate on validation set. Choose p-value threshold that gives highest AUC.**


4. **Evaluate best model on test set, possibly after tuning decision thresholds on validation set.**

Note that we explicitly find Healthy-specific sequences in the featurization.

# Analyze exact matches model performance on validation set

In [1]:
from IPython.display import display, Markdown
from malid import config, logger, io
from malid.external import model_evaluation
from malid.datamodels import (
    combine_classification_option_names,
    TargetObsColumnEnum,
    GeneLocus,
)
from malid.trained_model_wrappers import ExactMatchesClassifier
import joblib
import gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
import matplotlib.ticker
from summarynb import show, indexed_csv, table, chunks

## Analyze on validation set

In [4]:
for gene_locus in config.gene_loci_used:
    for target_obs_col in [
        TargetObsColumnEnum.disease
    ]:  # config.classification_targets:
        models_base_dir = ExactMatchesClassifier._get_model_base_dir(
            gene_locus=gene_locus, target_obs_column=target_obs_col
        )  # should already exist

        output_base_dir = (
            config.paths.exact_matches_output_dir
            / gene_locus.name
            / combine_classification_option_names(target_obs_col)
        )  # might not yet exist
        output_base_dir.mkdir(parents=True, exist_ok=True)  # create if needed

        model_output_prefix = models_base_dir / "train_smaller_model"
        results_output_prefix = output_base_dir / "train_smaller_model"

        try:
            logger.info(
                f"{gene_locus}, {target_obs_col} from {model_output_prefix} to {results_output_prefix}"
            )

            ## Load and summarize
            experiment_set = model_evaluation.ExperimentSet.load_from_disk(
                output_prefix=model_output_prefix
            )

            # Remove global fold (we trained global fold model, but now get evaluation scores on cross-validation folds only)
            # TODO: make kdict support: del self.model_outputs[:, fold_id]
            for key in experiment_set.model_outputs[:, -1].keys():
                logger.debug(f"Removing {key} (global fold)")
                del experiment_set.model_outputs[key]

            experiment_set_global_performance = experiment_set.summarize()
            experiment_set_global_performance.export_all_models(
                func_generate_classification_report_fname=lambda model_name: f"{results_output_prefix}.classification_report.{model_name}.txt",
                func_generate_confusion_matrix_fname=lambda model_name: f"{results_output_prefix}.confusion_matrix.{model_name}.png",
                dpi=72,
            )
            combined_stats = (
                experiment_set_global_performance.get_model_comparison_stats()
            )
            combined_stats.to_csv(
                f"{results_output_prefix}.compare_model_scores.tsv",
                sep="\t",
            )
            print(gene_locus, target_obs_col)
            display(combined_stats)

        except Exception as err:
            logger.exception(f"{gene_locus}, {target_obs_col} failed with error: {err}")

2022-12-26 08:39:48,541 - analyze_exact_matches_models.ipynb - INFO - GeneLocus.BCR, TargetObsColumnEnum.disease from /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/exact_matches/BCR/disease/train_smaller_model to /users/maximz/code/boyd-immune-repertoire-classification/out/exact_matches/BCR/disease/train_smaller_model


GeneLocus.BCR TargetObsColumnEnum.disease


,ROC-AUC (weighted OvO) per fold,ROC-AUC (macro OvO) per fold,au-PRC (weighted OvO) per fold,au-PRC (macro OvO) per fold,Accuracy per fold,MCC per fold,Accuracy global,MCC global,Accuracy per fold with abstention,MCC per fold with abstention,Unknown/abstention proportion per fold with abstention,Accuracy global with abstention,MCC global with abstention,Unknown/abstention proportion global with abstention,Abstention label global with abstention,sample_size,n_abstentions,sample_size including abstentions,abstention_rate,missing_classes
linearsvm_ovr,0.932 +/- 0.034 (in 3 folds),0.931 +/- 0.047 (in 3 folds),0.924 +/- 0.049 (in 3 folds),0.927 +/- 0.055 (in 3 folds),0.810 +/- 0.028 (in 3 folds),0.721 +/- 0.048 (in 3 folds),0.810,0.721,0.542 +/- 0.041 (in 3 folds),0.465 +/- 0.048 (in 3 folds),0.331 +/- 0.032 (in 3 folds),0.542,0.464,0.331,Unknown,216,107,323,0.331269,False
lasso_multiclass,0.928 +/- 0.027 (in 3 folds),0.928 +/- 0.039 (in 3 folds),0.920 +/- 0.042 (in 3 folds),0.924 +/- 0.048 (in 3 folds),0.787 +/- 0.022 (in 3 folds),0.686 +/- 0.043 (in 3 folds),0.787,0.685,0.527 +/- 0.035 (in 3 folds),0.439 +/- 0.040 (in 3 folds),0.331 +/- 0.032 (in 3 folds),0.526,0.438,0.331,Unknown,216,107,323,0.331269,False
lasso_cv,0.915 +/- 0.033 (in 3 folds),0.908 +/- 0.054 (in 3 folds),0.906 +/- 0.042 (in 3 folds),0.901 +/- 0.064 (in 3 folds),0.720 +/- 0.151 (in 3 folds),0.596 +/- 0.197 (in 3 folds),0.690,0.532,0.517 +/- 0.051 (in 3 folds),0.410 +/- 0.053 (in 3 folds),0.251 +/- 0.220 (in 3 folds),0.517,0.357,0.251,Unknown,242,81,323,0.250774,False
rf_multiclass,0.901 +/- 0.039 (in 3 folds),0.901 +/- 0.047 (in 3 folds),0.895 +/- 0.045 (in 3 folds),0.899 +/- 0.051 (in 3 folds),0.722 +/- 0.083 (in 3 folds),0.594 +/- 0.126 (in 3 folds),0.712,0.569,0.581 +/- 0.043 (in 3 folds),0.449 +/- 0.025 (in 3 folds),0.184 +/- 0.148 (in 3 folds),0.582,0.423,0.183,Unknown,264,59,323,0.182663,False
dummy_stratified,0.555 +/- 0.021 (in 3 folds),0.545 +/- 0.018 (in 3 folds),0.537 +/- 0.012 (in 3 folds),0.531 +/- 0.011 (in 3 folds),0.428 +/- 0.031 (in 3 folds),0.133 +/- 0.041 (in 3 folds),0.429,0.136,0.366 +/- 0.059 (in 3 folds),0.123 +/- 0.033 (in 3 folds),0.147 +/- 0.092 (in 3 folds),0.365,0.123,0.149,Unknown,275,48,323,0.148607,False
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.482 +/- 0.025 (in 3 folds),0.000 +/- 0.000 (in 3 folds),0.480,0.000,0.409 +/- 0.086 (in 3 folds),0.061 +/- 0.032 (in 3 folds),0.147 +/- 0.190 (in 3 folds),0.409,0.038,0.149,Unknown,275,48,323,0.148607,True


2022-12-26 08:39:51,349 - analyze_exact_matches_models.ipynb - INFO - GeneLocus.TCR, TargetObsColumnEnum.disease from /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/exact_matches/TCR/disease/train_smaller_model to /users/maximz/code/boyd-immune-repertoire-classification/out/exact_matches/TCR/disease/train_smaller_model


GeneLocus.TCR TargetObsColumnEnum.disease


,ROC-AUC (weighted OvO) per fold,ROC-AUC (macro OvO) per fold,au-PRC (weighted OvO) per fold,au-PRC (macro OvO) per fold,Accuracy per fold,MCC per fold,Accuracy global,MCC global,sample_size,n_abstentions,sample_size including abstentions,abstention_rate,missing_classes
linearsvm_ovr,0.877 +/- 0.017 (in 3 folds),0.872 +/- 0.016 (in 3 folds),0.882 +/- 0.008 (in 3 folds),0.879 +/- 0.007 (in 3 folds),0.668 +/- 0.037 (in 3 folds),0.502 +/- 0.068 (in 3 folds),0.668,0.501,274,0,274,0.0,False
lasso_multiclass,0.843 +/- 0.009 (in 3 folds),0.832 +/- 0.014 (in 3 folds),0.847 +/- 0.016 (in 3 folds),0.840 +/- 0.020 (in 3 folds),0.660 +/- 0.021 (in 3 folds),0.481 +/- 0.033 (in 3 folds),0.661,0.481,274,0,274,0.0,False
rf_multiclass,0.832 +/- 0.042 (in 3 folds),0.820 +/- 0.042 (in 3 folds),0.834 +/- 0.034 (in 3 folds),0.829 +/- 0.036 (in 3 folds),0.664 +/- 0.049 (in 3 folds),0.495 +/- 0.079 (in 3 folds),0.664,0.495,274,0,274,0.0,False
lasso_cv,0.825 +/- 0.045 (in 3 folds),0.816 +/- 0.050 (in 3 folds),0.832 +/- 0.037 (in 3 folds),0.828 +/- 0.040 (in 3 folds),0.657 +/- 0.040 (in 3 folds),0.476 +/- 0.062 (in 3 folds),0.657,0.475,274,0,274,0.0,False
dummy_stratified,0.536 +/- 0.009 (in 3 folds),0.528 +/- 0.004 (in 3 folds),0.531 +/- 0.004 (in 3 folds),0.526 +/- 0.002 (in 3 folds),0.401 +/- 0.015 (in 3 folds),0.094 +/- 0.021 (in 3 folds),0.401,0.094,274,0,274,0.0,False
dummy_most_frequent,0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.500 +/- 0.000 (in 3 folds),0.474 +/- 0.003 (in 3 folds),0.000 +/- 0.000 (in 3 folds),0.474,0.000,274,0,274,0.0,True


In [5]:
model_names = combined_stats.index[
    ~combined_stats.index.str.startswith("dummy_")
].tolist()
model_names

['linearsvm_ovr', 'lasso_multiclass', 'rf_multiclass', 'lasso_cv']

# Evaluate on test set

In [6]:
target_obs_column = TargetObsColumnEnum.disease
featurized_all = {}
for gene_locus in config.gene_loci_used:
    print(gene_locus)
    results_output_dir = (
        config.paths.exact_matches_output_dir
        / gene_locus.name
        / combine_classification_option_names(target_obs_column)
    )  # already created above when analyzing validation set

    results = model_evaluation.ExperimentSet()
    for fold_id in config.cross_validation_fold_ids:
        adata_test = io.load_fold_embeddings(
            fold_id=fold_id,
            fold_label="test",
            gene_locus=gene_locus,
            target_obs_column=target_obs_column,
        )

        # load validation set for tuning
        adata_validation = io.load_fold_embeddings(
            fold_id=fold_id,
            fold_label="validation",
            gene_locus=gene_locus,
            target_obs_column=target_obs_column,
        )

        for model_name in model_names:
            clf = ExactMatchesClassifier(
                fold_id=fold_id,
                model_name=model_name,
                fold_label_train="train_smaller",
                target_obs_column=target_obs_column,
                gene_locus=gene_locus,
            )

            # Featurization may vary by model, because p-value threshold may vary by model
            featurized: model_evaluation.FeaturizedData = clf.featurize(adata_test)
            # store for later analysis of featurized.extras
            featurized_all[(gene_locus, fold_id, model_name)] = featurized

            def _make_result(
                name: str,
                model: model_evaluation.Classifier,
                featurized_test: model_evaluation.FeaturizedData,
            ):
                # For curiosity's sake, what does the model predict for an input row of all 0s?
                row_of_0s: pd.DataFrame = (
                    pd.Series(1, index=model.feature_names_in_).to_frame().T
                )
                row_of_0s_predict = model.predict(row_of_0s)[0]
                row_of_0s_predict_proba = (
                    pd.DataFrame(model.predict_proba(row_of_0s), columns=model.classes_)
                    .iloc[0]
                    .apply(lambda p: f"{p:0.2%}")
                    .to_dict()
                )
                logger.info(
                    f"For a row of all zeros ({row_of_0s.shape}), model {name} on fold {fold_id}, {gene_locus}, {target_obs_column} predicts: {row_of_0s_predict} with probabilities {row_of_0s_predict_proba}"
                )

                # Create performance object
                return model_evaluation.ModelSingleFoldPerformance(
                    model_name=name,
                    fold_id=fold_id,
                    y_true=featurized_test.y,
                    clf=model,
                    X_test=featurized_test.X,
                    fold_label_train="train_smaller",
                    fold_label_test="test",
                    test_metadata=featurized_test.metadata,
                    test_abstentions=featurized_test.abstained_sample_y,
                    test_abstention_metadata=featurized_test.abstained_sample_metadata,
                )

            results.add(_make_result(model_name, clf, featurized))

            try:
                # tune on validation set
                clf_tuned = clf.tune_model_decision_thresholds_to_validation_set(
                    validation_set=adata_validation
                )

                # save out tuned model
                joblib.dump(
                    clf_tuned,
                    clf_tuned.models_base_dir
                    / f"train_smaller_model.{model_name}.decision_thresholds_tuned.{fold_id}.joblib",
                )

                # add tuned model performance
                results.add(
                    _make_result(
                        f"{model_name}_decision_thresholds_tuned", clf_tuned, featurized
                    )
                )
            except Exception as err:
                logger.warning(
                    f"Failed to tune {model_name} on validation set for fold {fold_id}, {gene_locus}, {target_obs_column}"
                )

        # Clear RAM
        del adata_test, adata_validation
        io.clear_cached_fold_embeddings()
        gc.collect()

    # summarize performance across folds and models
    summary = results.summarize()
    combined_stats = summary.get_model_comparison_stats()
    display(combined_stats)

    fname = (
        results_output_dir
        / f"train_smaller_model.compare_model_scores.test_set_performance.tsv"
    )
    combined_stats.to_csv(fname, sep="\t")

    summary.export_all_models(
        func_generate_classification_report_fname=lambda model_name: results_output_dir
        / f"train_smaller_model.test_set_performance.{model_name}.classification_report.txt",
        func_generate_confusion_matrix_fname=lambda model_name: results_output_dir
        / f"train_smaller_model.test_set_performance.{model_name}.confusion_matrix.png",
        dpi=72,
    )

2022-12-26 08:39:55,916 - malid.external.scratch_cache - INFO - Caching network file to local machine cache: /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/embedded/unirep_fine_tuned/anndatas_scaled/BCR/fold.0.test.h5ad -> /srv/scratch/maximz/cache/d23bdbcb1cb8d1c3007da595727fbbab8cc86779ab8967868314f2da.0.test.h5ad


GeneLocus.BCR
Only considering the two last: ['.test', '.h5ad'].
Only considering the two last: ['.test', '.h5ad'].


2022-12-26 08:47:11,809 - malid.helpers - WARNING - Not marking any sequences as confident_sequence because those annotations weren't found: [Errno 2] No such file or directory: '/users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/embedded/unirep_fine_tuned/anndatas_scaled/BCR/neighborhoods/knn_neighborhood.0.test.IGHG.embedded_onto.train_smaller.combined_data.filtered.tsv.gz'
2022-12-26 08:47:43,709 - malid.external.scratch_cache - INFO - Reading network file from local machine cache: /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/embedded/unirep_fine_tuned/anndatas_scaled/BCR/fold.0.validation.h5ad -> /srv/scratch/maximz/cache/1a7cd102c4a71eb472da3636203a91984ec407adbc0812ffba2c07c5.0.validation.h5ad


Only considering the two last: ['.validation', '.h5ad'].
Only considering the two last: ['.validation', '.h5ad'].


2022-12-26 08:49:19,132 - malid.helpers - WARNING - Not marking any sequences as confident_sequence because those annotations weren't found: [Errno 2] No such file or directory: '/users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/embedded/unirep_fine_tuned/anndatas_scaled/BCR/neighborhoods/knn_neighborhood.0.validation.IGHG.embedded_onto.train_smaller.combined_data.filtered.tsv.gz'
2022-12-26 08:50:20,948 - analyze_exact_matches_models.ipynb - INFO - For a row of all zeros ((1, 4)), model linearsvm_ovr on fold 0, GeneLocus.BCR, TargetObsColumnEnum.disease predicts: HIV with probabilities {'Covid19': '0.32%', 'HIV': '99.68%', 'Healthy/Background': '0.00%', 'Lupus': '0.00%'}
2022-12-26 08:50:42,361 - analyze_exact_matches_models.ipynb - INFO - For a row of all zeros ((1, 4)), model linearsvm_ovr_decision_thresholds_tuned on fold 0, GeneLocus.BCR, TargetObsColumnEnum.disease predicts: HIV with probabilities {'Covid19': '0.16%', 'HIV': '74.02%', 'Healthy/Backgrou

...

Only considering the two last: ['.test', '.h5ad'].
Only considering the two last: ['.test', '.h5ad'].


2022-12-26 10:02:05,328 - malid.helpers - WARNING - Not marking any sequences as confident_sequence because those annotations weren't found: [Errno 2] No such file or directory: '/users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/embedded/unirep_fine_tuned/anndatas_scaled/TCR/neighborhoods/knn_neighborhood.2.test.TCRB.embedded_onto.train_smaller.combined_data.filtered.tsv.gz'
2022-12-26 10:02:39,926 - malid.external.scratch_cache - INFO - Reading network file from local machine cache: /users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/embedded/unirep_fine_tuned/anndatas_scaled/TCR/fold.2.validation.h5ad -> /srv/scratch/maximz/cache/1ca7b8d5a7e7747e7f0bda4a4f3cb1e09d543670afe9d48fe77350c1.2.validation.h5ad


Only considering the two last: ['.validation', '.h5ad'].
Only considering the two last: ['.validation', '.h5ad'].


2022-12-26 10:05:22,843 - malid.helpers - WARNING - Not marking any sequences as confident_sequence because those annotations weren't found: [Errno 2] No such file or directory: '/users/maximz/code/boyd-immune-repertoire-classification/data/data_v_20221224/embedded/unirep_fine_tuned/anndatas_scaled/TCR/neighborhoods/knn_neighborhood.2.validation.TCRB.embedded_onto.train_smaller.combined_data.filtered.tsv.gz'
2022-12-26 10:06:22,485 - analyze_exact_matches_models.ipynb - INFO - For a row of all zeros ((1, 4)), model linearsvm_ovr on fold 2, GeneLocus.TCR, TargetObsColumnEnum.disease predicts: HIV with probabilities {'Covid19': '40.01%', 'HIV': '55.42%', 'Healthy/Background': '1.45%', 'Lupus': '3.12%'}
2022-12-26 10:06:49,448 - analyze_exact_matches_models.ipynb - INFO - For a row of all zeros ((1, 4)), model linearsvm_ovr_decision_thresholds_tuned on fold 2, GeneLocus.TCR, TargetObsColumnEnum.disease predicts: HIV with probabilities {'Covid19': '29.74%', 'HIV': '37.98%', 'Healthy/Backgr

,ROC-AUC (weighted OvO) per fold,ROC-AUC (macro OvO) per fold,au-PRC (weighted OvO) per fold,au-PRC (macro OvO) per fold,Accuracy per fold,MCC per fold,Accuracy global,MCC global,sample_size,n_abstentions,...,MCC per fold with abstention,Unknown/abstention proportion per fold with abstention,ROC-AUC (weighted OvO) per fold with abstention,ROC-AUC (macro OvO) per fold with abstention,au-PRC (weighted OvO) per fold with abstention,au-PRC (macro OvO) per fold with abstention,Accuracy global with abstention,MCC global with abstention,Unknown/abstention proportion global with abstention,Abstention label global with abstention
linearsvm_ovr,0.842 +/- 0.054 (in 3 folds),0.834 +/- 0.061 (in 3 folds),0.846 +/- 0.050 (in 3 folds),0.842 +/- 0.057 (in 3 folds),0.635 +/- 0.049 (in 3 folds),0.445 +/- 0.079 (in 3 folds),0.635,0.443,414,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
linearsvm_ovr_decision_thresholds_tuned,0.842 +/- 0.054 (in 3 folds),0.834 +/- 0.061 (in 3 folds),0.846 +/- 0.050 (in 3 folds),0.842 +/- 0.057 (in 3 folds),0.640 +/- 0.071 (in 3 folds),0.499 +/- 0.087 (in 3 folds),0.640,0.490,414,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rf_multiclass,0.824 +/- 0.012 (in 3 folds),0.813 +/- 0.017 (in 3 folds),0.813 +/- 0.025 (in 3 folds),0.806 +/- 0.034 (in 3 folds),0.636 +/- 0.004 (in 3 folds),0.453 +/- 0.011 (in 3 folds),0.636,0.443,412,2,...,0.450 +/- 0.014 (in 3 folds),0.014 +/- 0.000 (in 1 folds),0.827 +/- 0.016 (in 2 folds),0.819 +/- 0.017 (in 2 folds),0.824 +/- 0.022 (in 2 folds),0.822 +/- 0.026 (in 2 folds),0.633,0.440,0.005,Unknown
rf_multiclass_decision_thresholds_tuned,0.824 +/- 0.012 (in 3 folds),0.813 +/- 0.017 (in 3 folds),0.813 +/- 0.025 (in 3 folds),0.806 +/- 0.034 (in 3 folds),0.663 +/- 0.026 (in 3 folds),0.502 +/- 0.030 (in 3 folds),0.663,0.496,412,2,...,0.500 +/- 0.030 (in 3 folds),0.014 +/- 0.000 (in 1 folds),0.827 +/- 0.016 (in 2 folds),0.819 +/- 0.017 (in 2 folds),0.824 +/- 0.022 (in 2 folds),0.822 +/- 0.026 (in 2 folds),0.659,0.494,0.005,Unknown
lasso_cv,0.819 +/- 0.012 (in 3 folds),0.809 +/- 0.015 (in 3 folds),0.826 +/- 0.012 (in 3 folds),0.819 +/- 0.014 (in 3 folds),0.629 +/- 0.029 (in 3 folds),0.435 +/- 0.044 (in 3 folds),0.629,0.427,412,2,...,0.433 +/- 0.046 (in 3 folds),0.014 +/- 0.000 (in 1 folds),0.826 +/- 0.001 (in 2 folds),0.817 +/- 0.003 (in 2 folds),0.833 +/- 0.001 (in 2 folds),0.827 +/- 0.005 (in 2 folds),0.626,0.425,0.005,Unknown
lasso_cv_decision_thresholds_tuned,0.819 +/- 0.012 (in 3 folds),0.809 +/- 0.015 (in 3 folds),0.826 +/- 0.012 (in 3 folds),0.819 +/- 0.014 (in 3 folds),0.651 +/- 0.012 (in 3 folds),0.483 +/- 0.024 (in 3 folds),0.650,0.469,412,2,...,0.481 +/- 0.025 (in 3 folds),0.014 +/- 0.000 (in 1 folds),0.826 +/- 0.001 (in 2 folds),0.817 +/- 0.003 (in 2 folds),0.833 +/- 0.001 (in 2 folds),0.827 +/- 0.005 (in 2 folds),0.647,0.467,0.005,Unknown
lasso_multiclass,0.815 +/- 0.034 (in 3 folds),0.807 +/- 0.035 (in 3 folds),0.817 +/- 0.037 (in 3 folds),0.812 +/- 0.038 (in 3 folds),0.599 +/- 0.004 (in 3 folds),0.391 +/- 0.006 (in 3 folds),0.600,0.389,412,2,...,0.389 +/- 0.004 (in 3 folds),0.014 +/- 0.000 (in 1 folds),0.816 +/- 0.048 (in 2 folds),0.811 +/- 0.049 (in 2 folds),0.820 +/- 0.052 (in 2 folds),0.816 +/- 0.052 (in 2 folds),0.597,0.387,0.005,Unknown
lasso_multiclass_decision_thresholds_tuned,0.815 +/- 0.034 (in 3 folds),0.807 +/- 0.035 (in 3 folds),0.817 +/- 0.037 (in 3 folds),0.812 +/- 0.038 (in 3 folds),0.631 +/- 0.012 (in 3 folds),0.460 +/- 0.012 (in 3 folds),0.631,0.453,412,2,...,0.458 +/- 0.012 (in 3 folds),0.014 +/- 0.000 (in 1 folds),0.816 +/- 0.048 (in 2 folds),0.811 +/- 0.049 (in 2 folds),0.820 +/- 0.052 (in 2 folds),0.816 +/- 0.052 (in 2 folds),0.628,0.450,0.005,Unknown


# Test set performance - summary

In [7]:
for gene_locus in config.gene_loci_used:
    display(Markdown(f"## {gene_locus} test set performance"))
    results_output_dir = (
        config.paths.exact_matches_output_dir
        / gene_locus.name
        / combine_classification_option_names(target_obs_column)
    )
    results_df = pd.read_csv(
        results_output_dir
        / "train_smaller_model.compare_model_scores.test_set_performance.tsv",
        sep="\t",
        index_col=0,
    )
    show(table(results_df))
    for model_name_chunks in chunks(results_df.index.sort_values(), 4):
        show(
            [
                [
                    results_output_dir
                    / f"train_smaller_model.test_set_performance.{model_name}.confusion_matrix.png"
                    for model_name in model_name_chunks
                ],
                [
                    results_output_dir
                    / f"train_smaller_model.test_set_performance.{model_name}.classification_report.txt"
                    for model_name in model_name_chunks
                ],
            ],
            headers=model_name_chunks,
        )

## GeneLocus.BCR test set performance

,ROC-AUC (weighted OvO) per fold,ROC-AUC (macro OvO) per fold,au-PRC (weighted OvO) per fold,au-PRC (macro OvO) per fold,Accuracy per fold,MCC per fold,Accuracy global,MCC global,Accuracy per fold with abstention,MCC per fold with abstention,Unknown/abstention proportion per fold with abstention,Accuracy global with abstention,MCC global with abstention,Unknown/abstention proportion global with abstention,Abstention label global with abstention,sample_size,n_abstentions,sample_size including abstentions,abstention_rate,missing_classes
linearsvm_ovr,0.947 +/- 0.010 (in 3 folds),0.953 +/- 0.009 (in 3 folds),0.939 +/- 0.010 (in 3 folds),0.946 +/- 0.009 (in 3 folds),0.821 +/- 0.006 (in 3 folds),0.738 +/- 0.021 (in 3 folds),0.822,0.739,0.566 +/- 0.055 (in 3 folds),0.489 +/- 0.051 (in 3 folds),0.311 +/- 0.062 (in 3 folds),0.567,0.488,0.310,Unknown,331,149,480,0.310417,False
linearsvm_ovr_decision_thresholds_tuned,0.947 +/- 0.010 (in 3 folds),0.953 +/- 0.009 (in 3 folds),0.939 +/- 0.010 (in 3 folds),0.946 +/- 0.009 (in 3 folds),0.832 +/- 0.044 (in 3 folds),0.765 +/- 0.040 (in 3 folds),0.831,0.762,0.573 +/- 0.059 (in 3 folds),0.513 +/- 0.048 (in 3 folds),0.311 +/- 0.062 (in 3 folds),0.573,0.511,0.310,Unknown,331,149,480,0.310417,False
lasso_multiclass,0.942 +/- 0.011 (in 3 folds),0.948 +/- 0.011 (in 3 folds),0.930 +/- 0.007 (in 3 folds),0.938 +/- 0.008 (in 3 folds),0.807 +/- 0.008 (in 3 folds),0.718 +/- 0.026 (in 3 folds),0.807,0.719,0.556 +/- 0.050 (in 3 folds),0.474 +/- 0.048 (in 3 folds),0.311 +/- 0.062 (in 3 folds),0.556,0.474,0.310,Unknown,331,149,480,0.310417,False
lasso_multiclass_decision_thresholds_tuned,0.942 +/- 0.011 (in 3 folds),0.948 +/- 0.011 (in 3 folds),0.930 +/- 0.007 (in 3 folds),0.938 +/- 0.008 (in 3 folds),0.815 +/- 0.051 (in 3 folds),0.740 +/- 0.052 (in 3 folds),0.813,0.734,0.560 +/- 0.042 (in 3 folds),0.495 +/- 0.024 (in 3 folds),0.311 +/- 0.062 (in 3 folds),0.560,0.493,0.310,Unknown,331,149,480,0.310417,False
lasso_cv,0.916 +/- 0.037 (in 3 folds),0.910 +/- 0.040 (in 3 folds),0.906 +/- 0.038 (in 3 folds),0.901 +/- 0.041 (in 3 folds),0.739 +/- 0.134 (in 3 folds),0.606 +/- 0.203 (in 3 folds),0.713,0.573,0.526 +/- 0.084 (in 3 folds),0.413 +/- 0.073 (in 3 folds),0.262 +/- 0.233 (in 3 folds),0.527,0.380,0.260,Unknown,355,125,480,0.260417,False
lasso_cv_decision_thresholds_tuned,0.916 +/- 0.037 (in 3 folds),0.910 +/- 0.040 (in 3 folds),0.906 +/- 0.038 (in 3 folds),0.901 +/- 0.041 (in 3 folds),0.786 +/- 0.043 (in 3 folds),0.687 +/- 0.072 (in 3 folds),0.777,0.679,0.573 +/- 0.152 (in 3 folds),0.497 +/- 0.124 (in 3 folds),0.262 +/- 0.233 (in 3 folds),0.575,0.483,0.260,Unknown,355,125,480,0.260417,False
rf_multiclass,0.876 +/- 0.040 (in 3 folds),0.872 +/- 0.050 (in 3 folds),0.862 +/- 0.044 (in 3 folds),0.862 +/- 0.050 (in 3 folds),0.697 +/- 0.070 (in 3 folds),0.553 +/- 0.105 (in 3 folds),0.693,0.539,0.573 +/- 0.058 (in 3 folds),0.428 +/- 0.045 (in 3 folds),0.171 +/- 0.128 (in 3 folds),0.573,0.407,0.173,Unknown,397,83,480,0.172917,False
rf_multiclass_decision_thresholds_tuned,0.876 +/- 0.040 (in 3 folds),0.872 +/- 0.050 (in 3 folds),0.862 +/- 0.044 (in 3 folds),0.862 +/- 0.050 (in 3 folds),0.730 +/- 0.070 (in 3 folds),0.612 +/- 0.107 (in 3 folds),0.725,0.602,0.601 +/- 0.065 (in 3 folds),0.484 +/- 0.046 (in 3 folds),0.171 +/- 0.128 (in 3 folds),0.600,0.476,0.173,Unknown,397,83,480,0.172917,False


lasso_cv,lasso_cv_decision_thresholds_tuned,lasso_multiclass,lasso_multiclass_decision_thresholds_tuned
,,,
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.916 +/- 0.037 (in 3 folds) ROC-AUC (macro OvO): 0.910 +/- 0.040 (in 3 folds) au-PRC (weighted OvO): 0.906 +/- 0.038 (in 3 folds) au-PRC (macro OvO): 0.901 +/- 0.041 (in 3 folds) Accuracy: 0.739 +/- 0.134 (in 3 folds) MCC: 0.606 +/- 0.203 (in 3 folds) Global scores without abstention: Accuracy: 0.713 MCC: 0.573 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.526 +/- 0.084 (in 3 folds) MCC: 0.413 +/- 0.073 (in 3 folds) Unknown/abstention proportion: 0.262 +/- 0.233 (in 3 folds) Global scores with abstention: Accuracy: 0.527 MCC: 0.380 Unknown/abstention proportion: 0.260 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.96 0.40 0.56 63 HIV 0.91 0.61 0.73 98 Healthy/Background 0.64 0.65 0.65 221 Lupus 0.61 0.26 0.36 98 Unknown 0.00 0.00 0.00 0 accuracy 0.53 480 macro avg 0.62 0.38 0.46 480 weighted avg 0.73 0.53 0.59 480,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.916 +/- 0.037 (in 3 folds) ROC-AUC (macro OvO): 0.910 +/- 0.040 (in 3 folds) au-PRC (weighted OvO): 0.906 +/- 0.038 (in 3 folds) au-PRC (macro OvO): 0.901 +/- 0.041 (in 3 folds) Accuracy: 0.786 +/- 0.043 (in 3 folds) MCC: 0.687 +/- 0.072 (in 3 folds) Global scores without abstention: Accuracy: 0.777 MCC: 0.679 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.573 +/- 0.152 (in 3 folds) MCC: 0.497 +/- 0.124 (in 3 folds) Unknown/abstention proportion: 0.262 +/- 0.233 (in 3 folds) Global scores with abstention: Accuracy: 0.575 MCC: 0.483 Unknown/abstention proportion: 0.260 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.96 0.37 0.53 63 HIV 0.82 0.84 0.83 98 Healthy/Background 0.84 0.55 0.66 221 Lupus 0.58 0.50 0.54 98 Unknown 0.00 0.00 0.00 0 accuracy 0.57 480 macro avg 0.64 0.45 0.51 480 weighted avg 0.80 0.57 0.65 480,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.942 +/- 0.011 (in 3 folds) ROC-AUC (macro OvO): 0.948 +/- 0.011 (in 3 folds) au-PRC (weighted OvO): 0.930 +/- 0.007 (in 3 folds) au-PRC (macro OvO): 0.938 +/- 0.008 (in 3 folds) Accuracy: 0.807 +/- 0.008 (in 3 folds) MCC: 0.718 +/- 0.026 (in 3 folds) Global scores without abstention: Accuracy: 0.807 MCC: 0.719 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.556 +/- 0.050 (in 3 folds) MCC: 0.474 +/- 0.048 (in 3 folds) Unknown/abstention proportion: 0.311 +/- 0.062 (in 3 folds) Global scores with abstention: Accuracy: 0.556 MCC: 0.474 Unknown/abstention proportion: 0.310 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 1.00 0.41 0.58 63 HIV 0.92 0.81 0.86 98 Healthy/Background 0.82 0.55 0.66 221 Lupus 0.57 0.42 0.48 98 Unknown 0.00 0.00 0.00 0 accuracy 0.56 480 macro avg 0.66 0.44 0.52 480 weighted avg 0.81 0.56 0.65 480,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.942 +/- 0.011 (in 3 folds) ROC-AUC (macro OvO): 0.948 +/- 0.011 (in 3 folds) au-PRC (weighted OvO): 0.930 +/- 0.007 (in 3 folds) au-PRC (macro OvO): 0.938 +/- 0.008 (in 3 folds) Accuracy: 0.815 +/- 0.051 (in 3 folds) MCC: 0.740 +/- 0.052 (in 3 folds) Global scores without abstention: Accuracy: 0.813 MCC: 0.734 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.560 +/- 0.042 (in 3 folds) MCC: 0.495 +/- 0.024 (in 3 folds) Unknown/abstention proportion: 0.311 +/- 0.062 (in 3 folds) Global scores with abstention: Accuracy: 0.560 MCC: 0.493 Unknown/abstention proportion: 0.310 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.93 0.43 0.59 63 

linearsvm_ovr,linearsvm_ovr_decision_thresholds_tuned,rf_multiclass,rf_multiclass_decision_thresholds_tuned
,,,
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.947 +/- 0.010 (in 3 folds) ROC-AUC (macro OvO): 0.953 +/- 0.009 (in 3 folds) au-PRC (weighted OvO): 0.939 +/- 0.010 (in 3 folds) au-PRC (macro OvO): 0.946 +/- 0.009 (in 3 folds) Accuracy: 0.821 +/- 0.006 (in 3 folds) MCC: 0.738 +/- 0.021 (in 3 folds) Global scores without abstention: Accuracy: 0.822 MCC: 0.739 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.566 +/- 0.055 (in 3 folds) MCC: 0.489 +/- 0.051 (in 3 folds) Unknown/abstention proportion: 0.311 +/- 0.062 (in 3 folds) Global scores with abstention: Accuracy: 0.567 MCC: 0.488 Unknown/abstention proportion: 0.310 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 1.00 0.41 0.58 63 HIV 0.91 0.85 0.88 98 Healthy/Background 0.83 0.56 0.67 221 Lupus 0.61 0.41 0.49 98 Unknown 0.00 0.00 0.00 0 accuracy 0.57 480 macro avg 0.67 0.44 0.52 480 weighted avg 0.82 0.57 0.66 480,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.947 +/- 0.010 (in 3 folds) ROC-AUC (macro OvO): 0.953 +/- 0.009 (in 3 folds) au-PRC (weighted OvO): 0.939 +/- 0.010 (in 3 folds) au-PRC (macro OvO): 0.946 +/- 0.009 (in 3 folds) Accuracy: 0.832 +/- 0.044 (in 3 folds) MCC: 0.765 +/- 0.040 (in 3 folds) Global scores without abstention: Accuracy: 0.831 MCC: 0.762 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.573 +/- 0.059 (in 3 folds) MCC: 0.513 +/- 0.048 (in 3 folds) Unknown/abstention proportion: 0.311 +/- 0.062 (in 3 folds) Global scores with abstention: Accuracy: 0.573 MCC: 0.511 Unknown/abstention proportion: 0.310 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.96 0.43 0.59 63 HIV 0.87 0.91 0.89 98 Healthy/Background 0.91 0.51 0.66 221 Lupus 0.60 0.47 0.53 98 Unknown 0.00 0.00 0.00 0 accuracy 0.57 480 macro avg 0.67 0.46 0.53 480 weighted avg 0.85 0.57 0.67 480,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.876 +/- 0.040 (in 3 folds) ROC-AUC (macro OvO): 0.872 +/- 0.050 (in 3 folds) au-PRC (weighted OvO): 0.862 +/- 0.044 (in 3 folds) au-PRC (macro OvO): 0.862 +/- 0.050 (in 3 folds) Accuracy: 0.697 +/- 0.070 (in 3 folds) MCC: 0.553 +/- 0.105 (in 3 folds) Global scores without abstention: Accuracy: 0.693 MCC: 0.539 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.573 +/- 0.058 (in 3 folds) MCC: 0.428 +/- 0.045 (in 3 folds) Unknown/abstention proportion: 0.171 +/- 0.128 (in 3 folds) Global scores with abstention: Accuracy: 0.573 MCC: 0.407 Unknown/abstention proportion: 0.173 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 1.00 0.43 0.60 63 HIV 0.88 0.60 0.72 98 Healthy/Background 0.65 0.73 0.69 221 Lupus 0.52 0.29 0.37 98 Unknown 0.00 0.00 0.00 0 accuracy 0.57 480 macro avg 0.61 0.41 0.47 480 weighted avg 0.71 0.57 0.62 480,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.876 +/- 0.040 (in 3 folds) ROC-AUC (macro OvO): 0.872 +/- 0.050 (in 3 folds) au-PRC (weighted OvO): 0.862 +/- 0.044 (in 3 folds) au-PRC (macro OvO): 0.862 +/- 0.050 (in 3 folds) Accuracy: 0.730 +/- 0.070 (in 3 folds) MCC: 0.612 +/- 0.107 (in 3 folds) Global scores without abstention: Accuracy: 0.725 MCC: 0.602 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.601 +/- 0.065 (in 3 folds) MCC: 0.484 +/- 0.046 (in 3 folds) Unknown/abstention proportion: 0.171 +/- 0.128 (in 3 folds) Global scores with abstention: Accuracy: 0.600 MCC: 0.476 Unknown/abstention proportion: 0.173 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.97 0.46 0.62

## GeneLocus.TCR test set performance

,ROC-AUC (weighted OvO) per fold,ROC-AUC (macro OvO) per fold,au-PRC (weighted OvO) per fold,au-PRC (macro OvO) per fold,Accuracy per fold,MCC per fold,Accuracy global,MCC global,sample_size,n_abstentions,sample_size including abstentions,abstention_rate,missing_classes,Accuracy per fold with abstention,MCC per fold with abstention,Unknown/abstention proportion per fold with abstention,ROC-AUC (weighted OvO) per fold with abstention,ROC-AUC (macro OvO) per fold with abstention,au-PRC (weighted OvO) per fold with abstention,au-PRC (macro OvO) per fold with abstention,Accuracy global with abstention,MCC global with abstention,Unknown/abstention proportion global with abstention,Abstention label global with abstention
linearsvm_ovr,0.842 +/- 0.054 (in 3 folds),0.834 +/- 0.061 (in 3 folds),0.846 +/- 0.050 (in 3 folds),0.842 +/- 0.057 (in 3 folds),0.635 +/- 0.049 (in 3 folds),0.445 +/- 0.079 (in 3 folds),0.635,0.443,414,0,414,0.000000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
linearsvm_ovr_decision_thresholds_tuned,0.842 +/- 0.054 (in 3 folds),0.834 +/- 0.061 (in 3 folds),0.846 +/- 0.050 (in 3 folds),0.842 +/- 0.057 (in 3 folds),0.640 +/- 0.071 (in 3 folds),0.499 +/- 0.087 (in 3 folds),0.640,0.490,414,0,414,0.000000,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rf_multiclass,0.824 +/- 0.012 (in 3 folds),0.813 +/- 0.017 (in 3 folds),0.813 +/- 0.025 (in 3 folds),0.806 +/- 0.034 (in 3 folds),0.636 +/- 0.004 (in 3 folds),0.453 +/- 0.011 (in 3 folds),0.636,0.443,412,2,414,0.004831,False,0.633 +/- 0.007 (in 3 folds),0.450 +/- 0.014 (in 3 folds),0.014 +/- 0.000 (in 1 folds),0.827 +/- 0.016 (in 2 folds),0.819 +/- 0.017 (in 2 folds),0.824 +/- 0.022 (in 2 folds),0.822 +/- 0.026 (in 2 folds),0.633,0.440,0.005,Unknown
rf_multiclass_decision_thresholds_tuned,0.824 +/- 0.012 (in 3 folds),0.813 +/- 0.017 (in 3 folds),0.813 +/- 0.025 (in 3 folds),0.806 +/- 0.034 (in 3 folds),0.663 +/- 0.026 (in 3 folds),0.502 +/- 0.030 (in 3 folds),0.663,0.496,412,2,414,0.004831,False,0.660 +/- 0.025 (in 3 folds),0.500 +/- 0.030 (in 3 folds),0.014 +/- 0.000 (in 1 folds),0.827 +/- 0.016 (in 2 folds),0.819 +/- 0.017 (in 2 folds),0.824 +/- 0.022 (in 2 folds),0.822 +/- 0.026 (in 2 folds),0.659,0.494,0.005,Unknown
lasso_cv,0.819 +/- 0.012 (in 3 folds),0.809 +/- 0.015 (in 3 folds),0.826 +/- 0.012 (in 3 folds),0.819 +/- 0.014 (in 3 folds),0.629 +/- 0.029 (in 3 folds),0.435 +/- 0.044 (in 3 folds),0.629,0.427,412,2,414,0.004831,False,0.626 +/- 0.031 (in 3 folds),0.433 +/- 0.046 (in 3 folds),0.014 +/- 0.000 (in 1 folds),0.826 +/- 0.001 (in 2 folds),0.817 +/- 0.003 (in 2 folds),0.833 +/- 0.001 (in 2 folds),0.827 +/- 0.005 (in 2 folds),0.626,0.425,0.005,Unknown
lasso_cv_decision_thresholds_tuned,0.819 +/- 0.012 (in 3 folds),0.809 +/- 0.015 (in 3 folds),0.826 +/- 0.012 (in 3 folds),0.819 +/- 0.014 (in 3 folds),0.651 +/- 0.012 (in 3 folds),0.483 +/- 0.024 (in 3 folds),0.650,0.469,412,2,414,0.004831,False,0.647 +/- 0.007 (in 3 folds),0.481 +/- 0.025 (in 3 folds),0.014 +/- 0.000 (in 1 folds),0.826 +/- 0.001 (in 2 folds),0.817 +/- 0.003 (in 2 folds),0.833 +/- 0.001 (in 2 folds),0.827 +/- 0.005 (in 2 folds),0.647,0.467,0.005,Unknown
lasso_multiclass,0.815 +/- 0.034 (in 3 folds),0.807 +/- 0.035 (in 3 folds),0.817 +/- 0.037 (in 3 folds),0.812 +/- 0.038 (in 3 folds),0.599 +/- 0.004 (in 3 folds),0.391 +/- 0.006 (in 3 folds),0.600,0.389,412,2,414,0.004831,False,0.597 +/- 0.007 (in 3 folds),0.389 +/- 0.004 (in 3 folds),0.014 +/- 0.000 (in 1 folds),0.816 +/- 0.048 (in 2 folds),0.811 +/- 0.049 (in 2 folds),0.820 +/- 0.052 (in 2 folds),0.816 +/- 0.052 (in 2 folds),0.597,0.387,0.005,Unknown
lasso_multiclass_decision_thresholds_tuned,0.815 +/- 0.034 (in 3 folds),0.807 +/- 0.035 (in 3 folds),0.817 +/- 0.037 (in 3 folds),0.812 +/- 0.038 (in 3 folds),0.631 +/- 0.012 (in 3 folds),0.460 +/- 0.012 (in 3 folds),0.631,0.453,412,2,414,0.004831,False,0.628 +/- 0.009 (in 3 folds),0.458 +/- 0.012 (in 3 folds),0.014 +/- 0.000 (in 1 folds),0.816 +/- 0.048 (in 2 folds),0.811 +/- 0.049 (in 2 fold

lasso_cv,lasso_cv_decision_thresholds_tuned,lasso_multiclass,lasso_multiclass_decision_thresholds_tuned
,,,
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.819 +/- 0.012 (in 3 folds) ROC-AUC (macro OvO): 0.809 +/- 0.015 (in 3 folds) au-PRC (weighted OvO): 0.826 +/- 0.012 (in 3 folds) au-PRC (macro OvO): 0.819 +/- 0.014 (in 3 folds) Accuracy: 0.629 +/- 0.029 (in 3 folds) MCC: 0.435 +/- 0.044 (in 3 folds) Global scores without abstention: Accuracy: 0.629 MCC: 0.427 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.626 +/- 0.031 (in 3 folds) MCC: 0.433 +/- 0.046 (in 3 folds) Unknown/abstention proportion: 0.014 +/- 0.000 (in 1 folds) ROC-AUC (weighted OvO): 0.826 +/- 0.001 (in 2 folds) ROC-AUC (macro OvO): 0.817 +/- 0.003 (in 2 folds) au-PRC (weighted OvO): 0.833 +/- 0.001 (in 2 folds) au-PRC (macro OvO): 0.827 +/- 0.005 (in 2 folds) Global scores with abstention: Accuracy: 0.626 MCC: 0.425 Unknown/abstention proportion: 0.005 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.48 0.22 0.31 58 HIV 0.85 0.47 0.61 98 Healthy/Background 0.60 0.92 0.72 194 Lupus 0.68 0.33 0.44 64 Unknown 0.00 0.00 0.00 0 accuracy 0.63 414 macro avg 0.52 0.39 0.42 414 weighted avg 0.65 0.63 0.59 414,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.819 +/- 0.012 (in 3 folds) ROC-AUC (macro OvO): 0.809 +/- 0.015 (in 3 folds) au-PRC (weighted OvO): 0.826 +/- 0.012 (in 3 folds) au-PRC (macro OvO): 0.819 +/- 0.014 (in 3 folds) Accuracy: 0.651 +/- 0.012 (in 3 folds) MCC: 0.483 +/- 0.024 (in 3 folds) Global scores without abstention: Accuracy: 0.650 MCC: 0.469 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.647 +/- 0.007 (in 3 folds) MCC: 0.481 +/- 0.025 (in 3 folds) Unknown/abstention proportion: 0.014 +/- 0.000 (in 1 folds) ROC-AUC (weighted OvO): 0.826 +/- 0.001 (in 2 folds) ROC-AUC (macro OvO): 0.817 +/- 0.003 (in 2 folds) au-PRC (weighted OvO): 0.833 +/- 0.001 (in 2 folds) au-PRC (macro OvO): 0.827 +/- 0.005 (in 2 folds) Global scores with abstention: Accuracy: 0.647 MCC: 0.467 Unknown/abstention proportion: 0.005 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.80 0.21 0.33 58 HIV 0.61 0.70 0.65 98 Healthy/Background 0.69 0.80 0.74 194 Lupus 0.54 0.48 0.51 64 Unknown 0.00 0.00 0.00 0 accuracy 0.65 414 macro avg 0.53 0.44 0.45 414 weighted avg 0.66 0.65 0.63 414,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.815 +/- 0.034 (in 3 folds) ROC-AUC (macro OvO): 0.807 +/- 0.035 (in 3 folds) au-PRC (weighted OvO): 0.817 +/- 0.037 (in 3 folds) au-PRC (macro OvO): 0.812 +/- 0.038 (in 3 folds) Accuracy: 0.599 +/- 0.004 (in 3 folds) MCC: 0.391 +/- 0.006 (in 3 folds) Global scores without abstention: Accuracy: 0.600 MCC: 0.389 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.597 +/- 0.007 (in 3 folds) MCC: 0.389 +/- 0.004 (in 3 folds) Unknown/abstention proportion: 0.014 +/- 0.000 (in 1 folds) ROC-AUC (weighted OvO): 0.816 +/- 0.048 (in 2 folds) ROC-AUC (macro OvO): 0.811 +/- 0.049 (in 2 folds) au-PRC (weighted OvO): 0.820 +/- 0.052 (in 2 folds) au-PRC (macro OvO): 0.816 +/- 0.052 (in 2 folds) Global scores with abstention: Accuracy: 0.597 MCC: 0.387 Unknown/abstention proportion: 0.005 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.27 0.31 0.29 58 HIV 0.73 0.47 0.57 98 Healthy/Background 0.64 0.82 0.72 194 Lupus 0.72 0.36 0.48 64 Unknown 0.00 0.00 0.00 0 accuracy 0.60 414 macro avg 0.47 0.39 0.41 414 weighted avg 0.62 0.60 0.59 414,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.815 +/- 0.034 (in 3 folds) ROC-AUC (macro OvO): 0.807 +/- 0.035 (in 3 folds) au-PRC (weighted OvO): 0.817 +/- 0.037 (in 3 folds) au-PRC (macro OvO): 0.812 +/-

linearsvm_ovr,linearsvm_ovr_decision_thresholds_tuned,rf_multiclass,rf_multiclass_decision_thresholds_tuned
,,,
Per-fold scores: ROC-AUC (weighted OvO): 0.842 +/- 0.054 (in 3 folds) ROC-AUC (macro OvO): 0.834 +/- 0.061 (in 3 folds) au-PRC (weighted OvO): 0.846 +/- 0.050 (in 3 folds) au-PRC (macro OvO): 0.842 +/- 0.057 (in 3 folds) Accuracy: 0.635 +/- 0.049 (in 3 folds) MCC: 0.445 +/- 0.079 (in 3 folds) Global scores: Accuracy: 0.635 MCC: 0.443 Global classification report: precision recall f1-score support Covid19 0.49 0.33 0.39 58 HIV 0.86 0.43 0.57 98 Healthy/Background 0.60 0.92 0.73 194 Lupus 0.82 0.36 0.50 64 accuracy 0.64 414 macro avg 0.69 0.51 0.55 414 weighted avg 0.68 0.64 0.61 414,Per-fold scores: ROC-AUC (weighted OvO): 0.842 +/- 0.054 (in 3 folds) ROC-AUC (macro OvO): 0.834 +/- 0.061 (in 3 folds) au-PRC (weighted OvO): 0.846 +/- 0.050 (in 3 folds) au-PRC (macro OvO): 0.842 +/- 0.057 (in 3 folds) Accuracy: 0.640 +/- 0.071 (in 3 folds) MCC: 0.499 +/- 0.087 (in 3 folds) Global scores: Accuracy: 0.640 MCC: 0.490 Global classification report: precision recall f1-score support Covid19 0.39 0.45 0.42 58 HIV 0.71 0.68 0.70 98 Healthy/Background 0.77 0.67 0.72 194 Lupus 0.49 0.66 0.56 64 accuracy 0.64 414 macro avg 0.59 0.61 0.60 414 weighted avg 0.66 0.64 0.65 414,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.824 +/- 0.012 (in 3 folds) ROC-AUC (macro OvO): 0.813 +/- 0.017 (in 3 folds) au-PRC (weighted OvO): 0.813 +/- 0.025 (in 3 folds) au-PRC (macro OvO): 0.806 +/- 0.034 (in 3 folds) Accuracy: 0.636 +/- 0.004 (in 3 folds) MCC: 0.453 +/- 0.011 (in 3 folds) Global scores without abstention: Accuracy: 0.636 MCC: 0.443 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.633 +/- 0.007 (in 3 folds) MCC: 0.450 +/- 0.014 (in 3 folds) Unknown/abstention proportion: 0.014 +/- 0.000 (in 1 folds) ROC-AUC (weighted OvO): 0.827 +/- 0.016 (in 2 folds) ROC-AUC (macro OvO): 0.819 +/- 0.017 (in 2 folds) au-PRC (weighted OvO): 0.824 +/- 0.022 (in 2 folds) au-PRC (macro OvO): 0.822 +/- 0.026 (in 2 folds) Global scores with abstention: Accuracy: 0.633 MCC: 0.440 Unknown/abstention proportion: 0.005 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.53 0.40 0.46 58 HIV 0.78 0.39 0.52 98 Healthy/Background 0.61 0.93 0.74 194 Lupus 0.87 0.31 0.46 64 Unknown 0.00 0.00 0.00 0 accuracy 0.63 414 macro avg 0.56 0.41 0.43 414 weighted avg 0.68 0.63 0.60 414,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.824 +/- 0.012 (in 3 folds) ROC-AUC (macro OvO): 0.813 +/- 0.017 (in 3 folds) au-PRC (weighted OvO): 0.813 +/- 0.025 (in 3 folds) au-PRC (macro OvO): 0.806 +/- 0.034 (in 3 folds) Accuracy: 0.663 +/- 0.026 (in 3 folds) MCC: 0.502 +/- 0.030 (in 3 folds) Global scores without abstention: Accuracy: 0.663 MCC: 0.496 Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.660 +/- 0.025 (in 3 folds) MCC: 0.500 +/- 0.030 (in 3 folds) Unknown/abstention proportion: 0.014 +/- 0.000 (in 1 folds) ROC-AUC (weighted OvO): 0.827 +/- 0.016 (in 2 folds) ROC-AUC (macro OvO): 0.819 +/- 0.017 (in 2 folds) au-PRC (weighted OvO): 0.824 +/- 0.022 (in 2 folds) au-PRC (macro OvO): 0.822 +/- 0.026 (in 2 folds) Global scores with abstention: Accuracy: 0.659 MCC: 0.494 Unknown/abstention proportion: 0.005 Abstention label: Unknown Global classification report with abstention: precision recall f1-score support Covid19 0.82 0.31 0.45 58 HIV 0.68 0.68 0.68 98 Healthy/Background 0.71 0.77 0.74 194 Lupus 0.47 0.59 0.53 64 Unknown 0.00 0.00 0.00 0 accuracy 0.66 414 macro avg 0.54 0.47 0.48 414 weighted avg 0.68 0.66 0.65 414


# Compare to CDR3 clustering

In [4]:
for gene_locus in config.gene_loci_used:
    results_output_prefix_alternative = (
        config.paths.convergent_clusters_output_dir
        / gene_locus.name
        / combine_classification_option_names(target_obs_column)
        / "train_smaller_model"
    )

    display(
        Markdown(
            f"## Compare to CDR3 clustering: {gene_locus}, {target_obs_column}, eval on test set, with and without tuning on validation set"
        )
    )
    try:
        ## All results in a table
        all_results = pd.read_csv(
            f"{results_output_prefix_alternative}.compare_model_scores.test_set_performance.tsv",
            sep="\t",
            index_col=0,
        )
        show(table(all_results), headers=["All results, sorted"])

        models_of_interest = all_results.index

        ## Confusion matrices
        for model_names in chunks(models_of_interest, 4):
            show(
                [
                    [
                        f"{results_output_prefix_alternative}.test_set_performance.{model_name}.classification_report.txt"
                        for model_name in model_names
                    ],
                    [
                        f"{results_output_prefix_alternative}.test_set_performance.{model_name}.confusion_matrix.png"
                        for model_name in model_names
                    ],
                ],
                max_width=500,
                headers=model_names,
            )
    except FileNotFoundError as err:
        logger.warning(f"Not run: {err}")

    display(Markdown("---"))

## Compare to CDR3 clustering: GeneLocus.BCR, TargetObsColumnEnum.disease, eval on test set, with and without tuning on validation set

linearsvm_ovr,linearsvm_ovr.decision_thresholds_tuned,lasso_multiclass,lasso_multiclass.decision_thresholds_tuned
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.929 +/- 0.007 (in 3 folds) ROC-AUC (macro OvO): 0.936 +/- 0.007 (in 3 folds) au-PRC (weighted OvO): 0.931 +/- 0.008 (in 3 folds) au-PRC (macro OvO): 0.939 +/- 0.008 (in 3 folds) Accuracy: 0.766 +/- 0.010 (in 3 folds) MCC: 0.655 +/- 0.012 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.765 MCC: 0.654 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.754 +/- 0.015 (in 3 folds) MCC: 0.642 +/- 0.020 (in 3 folds) Unknown/abstention proportion: 0.022 +/- 0.022 (in 2 folds) ROC-AUC (weighted OvO): 0.931 +/- 0.000 (in 1 folds) ROC-AUC (macro OvO): 0.940 +/- 0.000 (in 1 folds) au-PRC (weighted OvO): 0.938 +/- 0.000 (in 1 folds) au-PRC (macro OvO): 0.946 +/- 0.000 (in 1 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.754 MCC: 0.640 Unknown/abstention proportion: 0.015 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.77 0.78 0.77 63 HIV 0.88 0.82 0.85 98 Healthy/Background 0.76 0.84 0.80 221 Lupus 0.64 0.48 0.55 98 Unknown 0.00 0.00 0.00 0 accuracy 0.75 480 macro avg 0.61 0.58 0.59 480 weighted avg 0.76 0.75 0.75 480,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.929 +/- 0.007 (in 3 folds) ROC-AUC (macro OvO): 0.936 +/- 0.007 (in 3 folds) au-PRC (weighted OvO): 0.931 +/- 0.008 (in 3 folds) au-PRC (macro OvO): 0.939 +/- 0.008 (in 3 folds) Accuracy: 0.757 +/- 0.007 (in 3 folds) MCC: 0.647 +/- 0.008 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.757 MCC: 0.641 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.746 +/- 0.014 (in 3 folds) MCC: 0.634 +/- 0.017 (in 3 folds) Unknown/abstention proportion: 0.022 +/- 0.022 (in 2 folds) ROC-AUC (weighted OvO): 0.931 +/- 0.000 (in 1 folds) ROC-AUC (macro OvO): 0.940 +/- 0.000 (in 1 folds) au-PRC (weighted OvO): 0.938 +/- 0.000 (in 1 folds) au-PRC (macro OvO): 0.946 +/- 0.000 (in 1 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.746 MCC: 0.628 Unknown/abstention proportion: 0.015 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.81 0.81 0.81 63 HIV 0.87 0.81 0.84 98 Healthy/Background 0.75 0.82 0.78 221 Lupus 0.60 0.48 0.53 98 Unknown 0.00 0.00 0.00 0 accuracy 0.75 480 macro avg 0.61 0.58 0.59 480 weighted avg 0.75 0.75 0.75 480,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.926 +/- 0.009 (in 3 folds) ROC-AUC (macro OvO): 0.933 +/- 0.010 (in 3 folds) au-PRC (weighted OvO): 0.927 +/- 0.014 (in 3 folds) au-PRC (macro OvO): 0.935 +/- 0.014 (in 3 folds) Accuracy: 0.761 +/- 0.010 (in 3 folds) MCC: 0.650 +/- 0.022 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.761 MCC: 0.649 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.744 +/- 0.031 (in 3 folds) MCC: 0.630 +/- 0.046 (in 3 folds) Unknown/abstention proportion: 0.034 +/- 0.030 (in 2 folds) ROC-AUC (weighted OvO): 0.931 +/- 0.000 (in 1 folds) ROC-AUC (macro OvO): 0.940 +/- 0.000 (in 1 folds) au-PRC (weighted OvO): 0.935 +/- 0.000 (in 1 folds) au-PRC (macro

lasso_cv,lasso_cv.decision_thresholds_tuned,elasticnet_cv,elasticnet_cv.decision_thresholds_tuned
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.909 +/- 0.014 (in 3 folds) ROC-AUC (macro OvO): 0.918 +/- 0.015 (in 3 folds) au-PRC (weighted OvO): 0.914 +/- 0.008 (in 3 folds) au-PRC (macro OvO): 0.923 +/- 0.008 (in 3 folds) Accuracy: 0.544 +/- 0.017 (in 3 folds) MCC: 0.317 +/- 0.047 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.543 MCC: 0.318 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.523 +/- 0.016 (in 3 folds) MCC: 0.267 +/- 0.052 (in 3 folds) Unknown/abstention proportion: 0.057 +/- 0.028 (in 2 folds) ROC-AUC (weighted OvO): 0.902 +/- 0.000 (in 1 folds) ROC-AUC (macro OvO): 0.914 +/- 0.000 (in 1 folds) au-PRC (weighted OvO): 0.912 +/- 0.000 (in 1 folds) au-PRC (macro OvO): 0.924 +/- 0.000 (in 1 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.523 MCC: 0.261 Unknown/abstention proportion: 0.037 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 1.00 0.03 0.06 63 HIV 1.00 0.30 0.46 98 Healthy/Background 0.50 0.96 0.66 221 Lupus 1.00 0.07 0.13 98 Unknown 0.00 0.00 0.00 0 accuracy 0.52 480 macro avg 0.70 0.27 0.26 480 weighted avg 0.77 0.52 0.43 480,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.909 +/- 0.014 (in 3 folds) ROC-AUC (macro OvO): 0.918 +/- 0.015 (in 3 folds) au-PRC (weighted OvO): 0.914 +/- 0.008 (in 3 folds) au-PRC (macro OvO): 0.923 +/- 0.008 (in 3 folds) Accuracy: 0.758 +/- 0.014 (in 3 folds) MCC: 0.644 +/- 0.024 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.758 MCC: 0.643 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.729 +/- 0.033 (in 3 folds) MCC: 0.606 +/- 0.047 (in 3 folds) Unknown/abstention proportion: 0.057 +/- 0.028 (in 2 folds) ROC-AUC (weighted OvO): 0.902 +/- 0.000 (in 1 folds) ROC-AUC (macro OvO): 0.914 +/- 0.000 (in 1 folds) au-PRC (weighted OvO): 0.912 +/- 0.000 (in 1 folds) au-PRC (macro OvO): 0.924 +/- 0.000 (in 1 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.729 MCC: 0.605 Unknown/abstention proportion: 0.037 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.85 0.75 0.80 63 HIV 0.88 0.76 0.81 98 Healthy/Background 0.70 0.87 0.78 221 Lupus 0.74 0.38 0.50 98 Unknown 0.00 0.00 0.00 0 accuracy 0.73 480 macro avg 0.64 0.55 0.58 480 weighted avg 0.77 0.73 0.73 480,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.909 +/- 0.011 (in 3 folds) ROC-AUC (macro OvO): 0.916 +/- 0.011 (in 3 folds) au-PRC (weighted OvO): 0.914 +/- 0.009 (in 3 folds) au-PRC (macro OvO): 0.923 +/- 0.009 (in 3 folds) Accuracy: 0.610 +/- 0.132 (in 3 folds) MCC: 0.390 +/- 0.278 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.612 MCC: 0.440 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.604 +/- 0.141 (in 3 folds) MCC: 0.379 +/- 0.297 (in 3 folds) Unknown/abstention proportion: 0.037 +/- 0.000 (in 1 folds) ROC-AUC (weighted OvO): 0.914 +/- 0.010 (in 2 folds) ROC-AUC (macro OvO): 0.921 +/- 0.010 (in 2 folds) au-PRC (weighted OvO): 0.918 +/- 0.008 (in 2 folds) au-PRC (macro OvO): 0.928 +/-

ridge_cv,ridge_cv.decision_thresholds_tuned,rf_multiclass,rf_multiclass.decision_thresholds_tuned
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.907 +/- 0.010 (in 3 folds) ROC-AUC (macro OvO): 0.917 +/- 0.011 (in 3 folds) au-PRC (weighted OvO): 0.914 +/- 0.006 (in 3 folds) au-PRC (macro OvO): 0.924 +/- 0.006 (in 3 folds) Accuracy: 0.576 +/- 0.065 (in 3 folds) MCC: 0.371 +/- 0.124 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.577 MCC: 0.383 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.572 +/- 0.068 (in 3 folds) MCC: 0.362 +/- 0.131 (in 3 folds) Unknown/abstention proportion: 0.009 +/- 0.004 (in 2 folds) ROC-AUC (weighted OvO): 0.913 +/- 0.000 (in 1 folds) ROC-AUC (macro OvO): 0.923 +/- 0.000 (in 1 folds) au-PRC (weighted OvO): 0.920 +/- 0.000 (in 1 folds) au-PRC (macro OvO): 0.930 +/- 0.000 (in 1 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.573 MCC: 0.374 Unknown/abstention proportion: 0.006 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 1.00 0.16 0.27 63 HIV 0.97 0.37 0.53 98 Healthy/Background 0.52 1.00 0.69 221 Lupus 1.00 0.09 0.17 98 Unknown 0.00 0.00 0.00 0 accuracy 0.57 480 macro avg 0.70 0.32 0.33 480 weighted avg 0.77 0.57 0.49 480,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.907 +/- 0.010 (in 3 folds) ROC-AUC (macro OvO): 0.917 +/- 0.011 (in 3 folds) au-PRC (weighted OvO): 0.914 +/- 0.006 (in 3 folds) au-PRC (macro OvO): 0.924 +/- 0.006 (in 3 folds) Accuracy: 0.761 +/- 0.033 (in 3 folds) MCC: 0.653 +/- 0.056 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.761 MCC: 0.652 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.756 +/- 0.036 (in 3 folds) MCC: 0.646 +/- 0.061 (in 3 folds) Unknown/abstention proportion: 0.009 +/- 0.004 (in 2 folds) ROC-AUC (weighted OvO): 0.913 +/- 0.000 (in 1 folds) ROC-AUC (macro OvO): 0.923 +/- 0.000 (in 1 folds) au-PRC (weighted OvO): 0.920 +/- 0.000 (in 1 folds) au-PRC (macro OvO): 0.930 +/- 0.000 (in 1 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.756 MCC: 0.645 Unknown/abstention proportion: 0.006 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.87 0.73 0.79 63 HIV 0.91 0.77 0.83 98 Healthy/Background 0.69 0.93 0.80 221 Lupus 0.80 0.37 0.50 98 Unknown 0.00 0.00 0.00 0 accuracy 0.76 480 macro avg 0.66 0.56 0.59 480 weighted avg 0.78 0.76 0.74 480,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.879 +/- 0.022 (in 3 folds) ROC-AUC (macro OvO): 0.886 +/- 0.023 (in 3 folds) au-PRC (weighted OvO): 0.872 +/- 0.028 (in 3 folds) au-PRC (macro OvO): 0.881 +/- 0.029 (in 3 folds) Accuracy: 0.723 +/- 0.038 (in 3 folds) MCC: 0.593 +/- 0.062 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.723 MCC: 0.589 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.701 +/- 0.052 (in 3 folds) MCC: 0.568 +/- 0.071 (in 3 folds) Unknown/abstention proportion: 0.045 +/- 0.046 (in 2 folds) ROC-AUC (weighted OvO): 0.900 +/- 0.000 (in 1 folds) ROC-AUC (macro OvO): 0.909 +/- 0.000 (in 1 folds) au-PRC (weighted OvO): 0.904 +/- 0.000 (in 1 folds) au-PRC (macro OvO): 0.915 +/-

xgboost,xgboost.decision_thresholds_tuned
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.870 +/- 0.026 (in 3 folds) ROC-AUC (macro OvO): 0.874 +/- 0.027 (in 3 folds) au-PRC (weighted OvO): 0.871 +/- 0.033 (in 3 folds) au-PRC (macro OvO): 0.880 +/- 0.034 (in 3 folds) Accuracy: 0.693 +/- 0.064 (in 3 folds) MCC: 0.547 +/- 0.103 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.693 MCC: 0.545 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.672 +/- 0.069 (in 3 folds) MCC: 0.524 +/- 0.103 (in 3 folds) Unknown/abstention proportion: 0.045 +/- 0.046 (in 2 folds) ROC-AUC (weighted OvO): 0.892 +/- 0.000 (in 1 folds) ROC-AUC (macro OvO): 0.897 +/- 0.000 (in 1 folds) au-PRC (weighted OvO): 0.905 +/- 0.000 (in 1 folds) au-PRC (macro OvO): 0.915 +/- 0.000 (in 1 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.673 MCC: 0.521 Unknown/abstention proportion: 0.029 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.83 0.71 0.77 63 HIV 0.84 0.76 0.80 98 Healthy/Background 0.69 0.76 0.72 221 Lupus 0.45 0.38 0.41 98 Unknown 0.00 0.00 0.00 0 accuracy 0.67 480 macro avg 0.56 0.52 0.54 480 weighted avg 0.69 0.67 0.68 480,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.870 +/- 0.026 (in 3 folds) ROC-AUC (macro OvO): 0.874 +/- 0.027 (in 3 folds) au-PRC (weighted OvO): 0.871 +/- 0.033 (in 3 folds) au-PRC (macro OvO): 0.880 +/- 0.034 (in 3 folds) Accuracy: 0.718 +/- 0.039 (in 3 folds) MCC: 0.599 +/- 0.056 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.719 MCC: 0.591 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.697 +/- 0.056 (in 3 folds) MCC: 0.576 +/- 0.066 (in 3 folds) Unknown/abstention proportion: 0.045 +/- 0.046 (in 2 folds) ROC-AUC (weighted OvO): 0.892 +/- 0.000 (in 1 folds) ROC-AUC (macro OvO): 0.897 +/- 0.000 (in 1 folds) au-PRC (weighted OvO): 0.905 +/- 0.000 (in 1 folds) au-PRC (macro OvO): 0.915 +/- 0.000 (in 1 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.698 MCC: 0.567 Unknown/abstention proportion: 0.029 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.82 0.78 0.80 63 HIV 0.80 0.83 0.81 98 Healthy/Background 0.75 0.73 0.74 221 Lupus 0.48 0.44 0.46 98 Unknown 0.00 0.00 0.00 0 accuracy 0.70 480 macro avg 0.57 0.56 0.56 480 weighted avg 0.72 0.70 0.71 480
,


...

ridge_cv,ridge_cv.decision_thresholds_tuned,linearsvm_ovr,linearsvm_ovr.decision_thresholds_tuned
Per-fold scores: ROC-AUC (weighted OvO): 0.894 +/- 0.019 (in 3 folds) ROC-AUC (macro OvO): 0.905 +/- 0.018 (in 3 folds) au-PRC (weighted OvO): 0.871 +/- 0.026 (in 3 folds) au-PRC (macro OvO): 0.889 +/- 0.024 (in 3 folds) Accuracy: 0.534 +/- 0.081 (in 3 folds) MCC: 0.214 +/- 0.215 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.534 MCC: 0.264 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.90 0.16 0.26 58 HIV 0.83 0.19 0.31 98 Healthy/Background 0.50 0.97 0.66 194 Lupus 1.00 0.06 0.12 64 accuracy 0.53 414 macro avg 0.81 0.35 0.34 414 weighted avg 0.71 0.53 0.44 414,Per-fold scores: ROC-AUC (weighted OvO): 0.894 +/- 0.019 (in 3 folds) ROC-AUC (macro OvO): 0.905 +/- 0.018 (in 3 folds) au-PRC (weighted OvO): 0.871 +/- 0.026 (in 3 folds) au-PRC (macro OvO): 0.889 +/- 0.024 (in 3 folds) Accuracy: 0.688 +/- 0.058 (in 3 folds) MCC: 0.578 +/- 0.075 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.688 MCC: 0.575 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.54 0.97 0.69 58 HIV 0.69 0.71 0.70 98 Healthy/Background 0.82 0.63 0.71 194 Lupus 0.62 0.58 0.60 64 accuracy 0.69 414 macro avg 0.67 0.72 0.68 414 weighted avg 0.72 0.69 0.69 414,Per-fold scores: ROC-AUC (weighted OvO): 0.893 +/- 0.012 (in 3 folds) ROC-AUC (macro OvO): 0.896 +/- 0.014 (in 3 folds) au-PRC (weighted OvO): 0.894 +/- 0.015 (in 3 folds) au-PRC (macro OvO): 0.902 +/- 0.015 (in 3 folds) Accuracy: 0.713 +/- 0.022 (in 3 folds) MCC: 0.577 +/- 0.028 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.713 MCC: 0.574 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.67 0.84 0.75 58 HIV 0.69 0.65 0.67 98 Healthy/Background 0.71 0.77 0.74 194 Lupus 0.89 0.52 0.65 64 accuracy 0.71 414 macro avg 0.74 0.70 0.70 414 weighted avg 0.73 0.71 0.71 414,Per-fold scores: ROC-AUC (weighted OvO): 0.893 +/- 0.012 (in 3 folds) ROC-AUC (macro OvO): 0.896 +/- 0.014 (in 3 folds) au-PRC (weighted OvO): 0.894 +/- 0.015 (in 3 folds) au-PRC (macro OvO): 0.902 +/- 0.015 (in 3 folds) Accuracy: 0.725 +/- 0.030 (in 3 folds) MCC: 0.617 +/- 0.037 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.725 MCC: 0.612 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.62 0.97 0.75 58 HIV 0.65 0.71 0.68 98 Healthy/Background 0.82 0.71 0.76 194 Lupus 0.77 0.56 0.65 64 accuracy 0.72 414 macro avg 0.71 0.74 0.71 414 weighted avg 0.74 0.72 0.72 414
,,,


lasso_multiclass,lasso_multiclass.decision_thresholds_tuned,xgboost,xgboost.decision_thresholds_tuned
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.885 +/- 0.005 (in 3 folds) ROC-AUC (macro OvO): 0.886 +/- 0.004 (in 3 folds) au-PRC (weighted OvO): 0.879 +/- 0.011 (in 3 folds) au-PRC (macro OvO): 0.884 +/- 0.011 (in 3 folds) Accuracy: 0.702 +/- 0.016 (in 3 folds) MCC: 0.575 +/- 0.023 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.702 MCC: 0.569 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.701 +/- 0.017 (in 3 folds) MCC: 0.573 +/- 0.021 (in 3 folds) Unknown/abstention proportion: 0.007 +/- 0.000 (in 1 folds) ROC-AUC (weighted OvO): 0.887 +/- 0.002 (in 2 folds) ROC-AUC (macro OvO): 0.889 +/- 0.002 (in 2 folds) au-PRC (weighted OvO): 0.885 +/- 0.007 (in 2 folds) au-PRC (macro OvO): 0.890 +/- 0.003 (in 2 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.700 MCC: 0.567 Unknown/abstention proportion: 0.002 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.57 0.86 0.69 58 HIV 0.68 0.68 0.68 98 Healthy/Background 0.76 0.74 0.75 194 Lupus 0.79 0.47 0.59 64 Unknown 0.00 0.00 0.00 0 accuracy 0.70 414 macro avg 0.56 0.55 0.54 414 weighted avg 0.72 0.70 0.70 414,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.885 +/- 0.005 (in 3 folds) ROC-AUC (macro OvO): 0.886 +/- 0.004 (in 3 folds) au-PRC (weighted OvO): 0.879 +/- 0.011 (in 3 folds) au-PRC (macro OvO): 0.884 +/- 0.011 (in 3 folds) Accuracy: 0.717 +/- 0.042 (in 3 folds) MCC: 0.600 +/- 0.055 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.717 MCC: 0.598 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.715 +/- 0.043 (in 3 folds) MCC: 0.598 +/- 0.056 (in 3 folds) Unknown/abstention proportion: 0.007 +/- 0.000 (in 1 folds) ROC-AUC (weighted OvO): 0.887 +/- 0.002 (in 2 folds) ROC-AUC (macro OvO): 0.889 +/- 0.002 (in 2 folds) au-PRC (weighted OvO): 0.885 +/- 0.007 (in 2 folds) au-PRC (macro OvO): 0.890 +/- 0.003 (in 2 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.715 MCC: 0.596 Unknown/abstention proportion: 0.002 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.57 0.86 0.68 58 HIV 0.68 0.73 0.71 98 Healthy/Background 0.81 0.71 0.76 194 Lupus 0.75 0.56 0.64 64 Unknown 0.00 0.00 0.00 0 accuracy 0.71 414 macro avg 0.56 0.57 0.56 414 weighted avg 0.73 0.71 0.72 414,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.883 +/- 0.003 (in 3 folds) ROC-AUC (macro OvO): 0.882 +/- 0.005 (in 3 folds) au-PRC (weighted OvO): 0.876 +/- 0.008 (in 3 folds) au-PRC (macro OvO): 0.880 +/- 0.006 (in 3 folds) Accuracy: 0.714 +/- 0.011 (in 3 folds) MCC: 0.572 +/- 0.015 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.714 MCC: 0.571 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.713 +/- 0.012 (in 3 folds) MCC: 0.570 +/- 0.016 (in 3 folds) Unknown/abstention proportion: 0.007 +/- 0.000 (in 1 folds) ROC-AUC (weighted OvO): 0.883 +/- 0.004 (in 2 folds) ROC-AUC (macro OvO): 0.881 +/- 0.006 (in 2 folds) au-PRC (weighted OvO): 0.881 +/- 0.004 (in 2 folds) au-PRC (macro OvO): 0.883

rf_multiclass,rf_multiclass.decision_thresholds_tuned,elasticnet_cv,elasticnet_cv.decision_thresholds_tuned
Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.881 +/- 0.011 (in 3 folds) ROC-AUC (macro OvO): 0.882 +/- 0.010 (in 3 folds) au-PRC (weighted OvO): 0.873 +/- 0.012 (in 3 folds) au-PRC (macro OvO): 0.877 +/- 0.010 (in 3 folds) Accuracy: 0.705 +/- 0.048 (in 3 folds) MCC: 0.558 +/- 0.077 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.705 MCC: 0.554 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.703 +/- 0.048 (in 3 folds) MCC: 0.556 +/- 0.077 (in 3 folds) Unknown/abstention proportion: 0.007 +/- 0.000 (in 1 folds) ROC-AUC (weighted OvO): 0.887 +/- 0.004 (in 2 folds) ROC-AUC (macro OvO): 0.888 +/- 0.004 (in 2 folds) au-PRC (weighted OvO): 0.880 +/- 0.001 (in 2 folds) au-PRC (macro OvO): 0.883 +/- 0.003 (in 2 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.703 MCC: 0.552 Unknown/abstention proportion: 0.002 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.64 0.55 0.59 58 HIV 0.67 0.57 0.62 98 Healthy/Background 0.71 0.85 0.77 194 Lupus 0.78 0.61 0.68 64 Unknown 0.00 0.00 0.00 0 accuracy 0.70 414 macro avg 0.56 0.52 0.53 414 weighted avg 0.70 0.70 0.70 414,Per-fold scores without abstention: ROC-AUC (weighted OvO): 0.881 +/- 0.011 (in 3 folds) ROC-AUC (macro OvO): 0.882 +/- 0.010 (in 3 folds) au-PRC (weighted OvO): 0.873 +/- 0.012 (in 3 folds) au-PRC (macro OvO): 0.877 +/- 0.010 (in 3 folds) Accuracy: 0.719 +/- 0.027 (in 3 folds) MCC: 0.593 +/- 0.041 (in 3 folds) Global scores without abstention using column name disease.separate_past_exposures: Accuracy: 0.719 MCC: 0.592 Global evaluation column name: disease.separate_past_exposures Per-fold scores with abstention (note that abstentions not included in probability-based scores): Accuracy: 0.718 +/- 0.028 (in 3 folds) MCC: 0.591 +/- 0.041 (in 3 folds) Unknown/abstention proportion: 0.007 +/- 0.000 (in 1 folds) ROC-AUC (weighted OvO): 0.887 +/- 0.004 (in 2 folds) ROC-AUC (macro OvO): 0.888 +/- 0.004 (in 2 folds) au-PRC (weighted OvO): 0.880 +/- 0.001 (in 2 folds) au-PRC (macro OvO): 0.883 +/- 0.003 (in 2 folds) Global scores with abstention using column name disease.separate_past_exposures: Accuracy: 0.717 MCC: 0.589 Unknown/abstention proportion: 0.002 Abstention label: Unknown Global evaluation column name: disease.separate_past_exposures Global classification report with abstention using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.57 0.81 0.67 58 HIV 0.66 0.61 0.63 98 Healthy/Background 0.79 0.78 0.79 194 Lupus 0.83 0.59 0.69 64 Unknown 0.00 0.00 0.00 0 accuracy 0.72 414 macro avg 0.57 0.56 0.56 414 weighted avg 0.73 0.72 0.72 414,Per-fold scores: ROC-AUC (weighted OvO): 0.866 +/- 0.056 (in 3 folds) ROC-AUC (macro OvO): 0.869 +/- 0.063 (in 3 folds) au-PRC (weighted OvO): 0.858 +/- 0.046 (in 3 folds) au-PRC (macro OvO): 0.867 +/- 0.055 (in 3 folds) Accuracy: 0.602 +/- 0.084 (in 3 folds) MCC: 0.374 +/- 0.151 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.601 MCC: 0.381 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.77 0.29 0.42 58 HIV 0.74 0.46 0.57 98 Healthy/Background 0.55 0.92 0.69 194 Lupus 1.00 0.12 0.22 64 accuracy 0.60 414 macro avg 0.77 0.45 0.48 414 weighted avg 0.70 0.60 0.55 414,Per-fold scores: ROC-AUC (weighted OvO): 0.866 +/- 0.056 (in 3 folds) ROC-AUC (macro OvO): 0.869 +/- 0.063 (in 3 folds) au-PRC (weighted OvO

lasso_cv,lasso_cv.decision_thresholds_tuned
Per-fold scores: ROC-AUC (weighted OvO): 0.860 +/- 0.032 (in 3 folds) ROC-AUC (macro OvO): 0.865 +/- 0.032 (in 3 folds) au-PRC (weighted OvO): 0.858 +/- 0.029 (in 3 folds) au-PRC (macro OvO): 0.868 +/- 0.030 (in 3 folds) Accuracy: 0.584 +/- 0.070 (in 3 folds) MCC: 0.343 +/- 0.128 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.585 MCC: 0.349 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.65 0.19 0.29 58 HIV 0.73 0.44 0.55 98 Healthy/Background 0.54 0.92 0.68 194 Lupus 1.00 0.14 0.25 64 accuracy 0.58 414 macro avg 0.73 0.42 0.44 414 weighted avg 0.67 0.58 0.53 414,Per-fold scores: ROC-AUC (weighted OvO): 0.860 +/- 0.032 (in 3 folds) ROC-AUC (macro OvO): 0.865 +/- 0.032 (in 3 folds) au-PRC (weighted OvO): 0.858 +/- 0.029 (in 3 folds) au-PRC (macro OvO): 0.868 +/- 0.030 (in 3 folds) Accuracy: 0.681 +/- 0.042 (in 3 folds) MCC: 0.569 +/- 0.053 (in 3 folds) Global scores using column name disease.separate_past_exposures: Accuracy: 0.681 MCC: 0.556 Global evaluation column name: disease.separate_past_exposures Global classification report using column name disease.separate_past_exposures: precision recall f1-score support Covid19 0.59 0.95 0.73 58 HIV 0.63 0.70 0.67 98 Healthy/Background 0.79 0.64 0.71 194 Lupus 0.62 0.53 0.57 64 accuracy 0.68 414 macro avg 0.66 0.71 0.67 414 weighted avg 0.70 0.68 0.68 414
,


---

# Which p values were chosen (varies by locus, model, and fold)? How many disease-associated sequences found?

In [9]:
for gene_locus in config.gene_loci_used:
    for fold_id in config.cross_validation_fold_ids:
        for model_name in model_names:
            clf = ExactMatchesClassifier(
                fold_id=fold_id,
                model_name=model_name,
                fold_label_train="train_smaller",
                target_obs_column=TargetObsColumnEnum.disease,
                gene_locus=gene_locus,
            )
            p_value = clf.p_value_threshold
            seqs = clf.sequences_with_fisher_result
            print(
                f"{gene_locus}, fold {fold_id}, {model_name}: p = {p_value}. Number of disease associated sequences: {(seqs <= p_value).sum().to_dict()}"
            )
        print()

GeneLocus.BCR, fold 0, linearsvm_ovr: p = 0.005. Number of disease associated sequences: {'Covid19': 46, 'HIV': 77, 'Healthy/Background': 21, 'Lupus': 18}
GeneLocus.BCR, fold 0, lasso_multiclass: p = 0.005. Number of disease associated sequences: {'Covid19': 46, 'HIV': 77, 'Healthy/Background': 21, 'Lupus': 18}
GeneLocus.BCR, fold 0, rf_multiclass: p = 0.01. Number of disease associated sequences: {'Covid19': 51, 'HIV': 219, 'Healthy/Background': 27, 'Lupus': 39}
GeneLocus.BCR, fold 0, lasso_cv: p = 0.1. Number of disease associated sequences: {'Covid19': 459, 'HIV': 2101, 'Healthy/Background': 149, 'Lupus': 1136}

GeneLocus.BCR, fold 1, linearsvm_ovr: p = 0.005. Number of disease associated sequences: {'Covid19': 50, 'HIV': 71, 'Healthy/Background': 25, 'Lupus': 17}
GeneLocus.BCR, fold 1, lasso_multiclass: p = 0.005. Number of disease associated sequences: {'Covid19': 50, 'HIV': 71, 'Healthy/Background': 25, 'Lupus': 17}
GeneLocus.BCR, fold 1, rf_multiclass: p = 0.005. Number of disea